# Basic Cashflow Computation
## Compute Constant Cashflow
This method will provide a constant cashflow of "**Initial Value**".

The cash flow starts after "**shift**" projection periods and is paid for "**length**" projection periods.

In [0]:
public static double[] GetConstantValues(this double initialValue, int shift, int length){

    var constant = new double[shift + length];
    
    for (var i = 0; i < length; i++)
        constant[shift + i] =  initialValue;
        
    return constant;
}

## Compute Linear Cashflow
This method provides a linearly increasing cash flow.

The amount of which is equal to the "**Initial Value**" plus any "**Values**" for the given time period.

The cash flow starts after "**shift**" projection periods and is paid for "**length**" projection periods.

In [0]:
public static double[] GetLinearValues(this double initialValue, int shift, int length, double[] values){

    // The cashflow start after "shift" periods. 
    // The cashflow length is "Lenght" periods.
    // The cashflow is the accumulation of "initialValue" + "value[i]"

    var linear = new double[shift + length];
    
    for (var i = 0; i < length; i++)
        linear[shift + i] =  initialValue + values.Take(i + 1).Aggregate((x, y) => x + y);
        
    return linear;
}

## Compute Compounded Cashflow
This method provides a geometrically increasing or decreasing cash flow.

The **amount** of the cash flow in the first period is equal to "**Initial Value**" * (1 + **Value**(t)). In subsequent periods, the cash flow is equal to the previous cash flow multiplied by (1 + **Value**(t)).

The cash flow starts after "**shift**" projection periods and is paid for "**length**" projection periods.

In [0]:
public static double[] GetCompoundValues(this double initialValue, int shift, int length, double[] values){
    
    var compound = new double[shift + length];

    for (var i = 0; i < length; i++)
        compound[shift + i] = initialValue * values.Take(i + 1).Select(x => 1 + x).Aggregate((x, y) => x * y);
             
    return compound;
}

# Bounded Cashflow Computation

## Compute Bounded Random Cashflows

This method provides random cash flow amounts, with values between the specified "**Minimum**" and "**Maximum**" boundaries.

The cash flow starts after the specified number of projection periods ("**shift**") and is paid for the specified number of projection periods ("**length**").

In [0]:
public static double[] GetBoundedRandomValues(this int shift, int length, double max, double min){
    var bounded = new double[length];
    var random = new Random();

    for (var i = shift; i < length; i++)
        bounded[i] = random.NextDouble() * (max - min) + min;
               
    return bounded;
}

Correlating Cash Flows for Risk Drivers in the Solvency Framework

Some cash flows for risk drivers in the Solvency framework can be correlated. This is done by using a correlation factor.

The correlated value of two correlated risks is calculated as follows:

Code snippet
$$
\text{Correlated Value} = \sqrt{\text{r1}^2 + \text{r2}^2 + 2ab\text{Corr}(\text{r1},\text{r2})}
$$
where:

$$r1$$ and $$r2$$ are the risk measures of the two correlated risks

$$\text{Corr}(\text{r1},\text{r2})$$ is the correlation factor between the two risk measures

$$a$$ and $$b$$ are the weights of the two risk measures

The correlation effect is the difference between the correlated value and the sum of the two risk measures:

$$
\text{Correlation Effect} = \text{Correlated Value} - \text{r1} - \text{r2}
$$

If both risks are the same, the correlation is assumed to be 1, and the correlation effect is 0. Therefore, there is no need to provide correlation factors of 1.

Multiple Risks

The correlated value of multiple risks is calculated as follows:

$$
\text{Correlated Value} = \sum_{i,j} \text{r}_i \text{r}_j \text{Corr}(\text{r}_i,\text{r}_j)
$$

where:

$$\text{r}_i$$ and $$\text{r}_j$$ are the risk measures of the individual risks
$$\text{Corr}(\text{r}_i,\text{r}_j)$$ is the correlation factor between the two risk measures
The correlation effect is calculated as follows:

$$\text{Correlation Effect} = \text{Correlated Value} - \sum_i \text{r}_i
$$

## Correlate Risk Cashflows
### Two risks
Some cashflows for Risk Drivers in the Solvency framework should allow for correlation between them. This is done by using a Correlation factor. 
$$
\text{Correlated Value} = \sqrt{r_1^2 + r_2^2 + 2 \cdot a \cdot b \cdot \text{Corr}(r_1,r_2)}
$$

where 
- $$r_1$$ and $$r_2$$ represent risk measure of 2 correlated risks
- $$Corr(r_1, r_2)$$ is correlation factor between such 2 risk measures
  
The total value, *in addition* to simple aggregation of *r_1* an *r_2* is expressed by
$$
\text{Correlation Effect} = \text{Correlated Value} - r_1 - r_2
$$

Note, if both risks are the same, the correlation is assumed to be 1, and the above formula for Correlation Effect simlifies to 0. Thus, there is no need to provide correlation factors of 1.

### Multiple risks
$$
\text{Correlated Value} = \sqrt{\sum_{ij} r_i \cdot r_j \cdot \text{Corr}(r_i,r_j)}
$$

$$
\text{Correlation Effect} = \text{Correlated Value} - \sum_{i}r_i
$$

Source: [Eiopa CEIOPS Correlation Parameters](https://register.eiopa.europa.eu/CEIOPS-Archive/Documents/Advices/CEIOPS-L2-Advice-Correlation-Parameters.pdf)

In [0]:
public static double Correlate(double[] r1, double[] r2, double[,] correlation){

    if (r1.Length != r2.Length)
        throw new System.Exception("Vectors of unequal lenghts!");

   return Math.Sqrt(r1.Select((x,i) => r2.Select((y,j) => x * y * correlation[i,j] ).Sum()).Sum());

}